Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/deployment/deploy-to-cloud/model-register-and-deploy.png)

# Register model and deploy as webservice in ACI

Following this notebook, you will:

 - Learn how to register a model in your Azure Machine Learning Workspace.
 - Deploy your model as a web service in an Azure Container Instance.

## Prerequisites

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, make sure you go through the [configuration notebook](../../../configuration.ipynb) to install the Azure Machine Learning Python SDK and create a workspace.

In [1]:
import azureml.core


# Check core SDK version number.
print('SDK version:', azureml.core.VERSION)

SDK version: 1.17.0


## Initialize workspace

Create a [Workspace](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.workspace%28class%29?view=azure-ml-py) object from your persisted configuration.

In [7]:
from azureml.core import Workspace


ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep='\n')

nabila-ml
nabila-amplify-rg
eastus
38d95926-fedd-400d-a5d4-60c70a647707


## Create trained model

For this example, we will train a small model on scikit-learn's [diabetes dataset](https://scikit-learn.org/stable/datasets/index.html#diabetes-dataset). 

In [2]:
import joblib
import sklearn
from torchvision import models
import torch
import torch.nn.functional as F
from sklearn.datasets import load_diabetes
from sklearn.linear_model import Ridge

#alexnet = models.alexnet(pretrained=True)
#joblib.dump(alexnet, 'alexnet_model.pkl')

In [3]:
!ls 

alexnet_model.pkl                       freshen-model-register-and-deploy.ipynb
banana.jpg                              image_serial.json
config.json                             imagenet_classes.txt
config.json.save                        outputfile.json
deploy-keras-model.ipynb                score.py
env.yml                                 testEndpoint.ipynb
fresh-rotten-model.h5


## Register model

Register a file or folder as a model by calling [Model.register()](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.model?view=azure-ml-py#register-workspace--model-path--model-name--tags-none--properties-none--description-none--datasets-none--model-framework-none--model-framework-version-none--child-paths-none-).

In addition to the content of the model file itself, your registered model will also store model metadata -- model description, tags, and framework information -- that will be useful when managing and deploying models in your workspace. Using tags, for instance, you can categorize your models and apply filters when listing models in your workspace. Also, marking this model with the scikit-learn framework will simplify deploying it as a web service, as we'll see later.

In [8]:
import os
from azureml.core import Model
from azureml.core.resource_configuration import ResourceConfiguration

In [ ]:
model = Model.register(workspace=ws,
                       model_name='alexnet',                # Name of the registered model in your workspace.
                       model_path='alexnet_model.pkl',  # Local file to upload and register as a model.

                       resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=1),
                       description='Predict image label',
                       tags={'area': 'image', 'type': 'recognition'}
                      )

In [9]:
model = Model(name='alexnet', workspace=ws)

In [10]:
print(model.name, model.id, model.version, sep='\t')

alexnet	alexnet:4	4


## Deploy model

Deploy your model as a web service using [Model.deploy()](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.model?view=azure-ml-py#deploy-workspace--name--models--inference-config--deployment-config-none--deployment-target-none-). Web services take one or more models, load them in an environment, and run them on one of several supported deployment targets. For more information on all your options when deploying models, see the [next steps](#Next-steps) section at the end of this notebook.

For this example, we will deploy your scikit-learn model to an Azure Container Instance (ACI).

### Use a custom environment

If you want more control over how your model is run, if it uses another framework, or if it has special runtime requirements, you can instead specify your own environment and scoring method. Custom environments can be used for any model you want to deploy.

Specify the model's runtime environment by creating an [Environment](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.environment%28class%29?view=azure-ml-py) object and providing the [CondaDependencies](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.conda_dependencies.condadependencies?view=azure-ml-py) needed by your model.

In [11]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies


environment = Environment('my-freshen-environment')
environment.python.conda_dependencies = CondaDependencies.create(pip_packages=[
    'azureml-defaults',
    'inference-schema[numpy-support]',
    'joblib',
    'numpy',
    'torch',
    'torchvision',
    'scikit-learn=={}'.format(sklearn.__version__)
])

In [12]:
%%writefile score.py
import os
import pickle
import json
import joblib
import numpy as np
import torch
from azureml.core.model import Model
from torchvision import transforms
from PIL import Image 

transform = transforms.Compose([            
 transforms.Resize(256),                    
 transforms.CenterCrop(224),                
 transforms.ToTensor(),                     
 transforms.Normalize(                  
 mean=[0.485, 0.456, 0.406],                
 std=[0.229, 0.224, 0.225]                  
 )])


def preprocess_data(img):
    print(f' Image shape in preprocess data: {img.shape}')
    img = Image.fromarray(img, 'RGB')
    print(f' Image type: {type(img)}')
    img_t = transform(img)
    batch_t = torch.unsqueeze(img_t, 0)
    return batch_t

def init():
    global model
    # retrieve the path to the model file using the model name
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'alexnet_model.pkl')
    model = joblib.load(model_path)

def run(raw_data):

    try:
        print('in the try, received data')
        data = np.array(json.loads(raw_data)['data'])
        print('after the read')
        # make prediction
        print(f'Length of the data list: {len(list(data))}')
        img = preprocess_data(data)
        print('after the preprocess')
        y = model(img)
        print('after inference')
        #prob = F.softmax(y, dim=1)[0]
        #_, ind = torch.topk(y, 1) 
        return y.tolist()[0]
    
    except Exception as e:
        error = str(e)
        return error

Overwriting score.py


When using a custom environment, you must also provide Python code for initializing and running your model. An example script is included with this notebook.

In [13]:
import os
import pickle
import json
import joblib
import numpy as np
from azureml.core.model import Model
from torchvision import transforms
from PIL import Image 

transform = transforms.Compose([            
 transforms.Resize(256),                    
 transforms.CenterCrop(224),                
 transforms.ToTensor(),                     
 transforms.Normalize(                  
 mean=[0.485, 0.456, 0.406],                
 std=[0.229, 0.224, 0.225]                  
 )])


def preprocess_data(img):
    print(f' Image shape in preprocess data: {img.shape}')
    img = Image.fromarray(img, 'RGB')
    print(f' Image type: {type(img)}')
    img_t = transform(img)
    batch_t = torch.unsqueeze(img_t, 0)
    return batch_t

def init():
    global model
    # retrieve the path to the model file using the model name
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'alexnet_model.pkl')
    model = joblib.load(model_path)

def run(raw_data):

    try:
        print('in the try, received data')
        model = joblib.load('alexnet_model.pkl')
        data = np.array(json.loads(raw_data)['data'])
        print('after the read')
        # make prediction
        print(f'Length of the data list: {len(list(data))}')
        img = preprocess_data(data)
        print('after the preprocess')
        y = model(img)
        print('after inference')
        prob = F.softmax(y, dim=1)[0]
        return y.tolist()[0]
    
    except Exception as e:
        error = str(e)
        return error

In [14]:
import cv2
import json

img = cv2.imread('banana.jpg')
input_data = {'data': img.tolist()}

In [15]:
#with open('image.txt', 'w') as file:
 #    file.write(json.dumps(input_data))

In [16]:
y = run(json.dumps(input_data))

in the try, received data
after the read
Length of the data list: 667
 Image shape in preprocess data: (667, 1000, 3)
 Image type: <class 'PIL.Image.Image'>
after the preprocess
after inference


In [17]:
print(y.shape)

AttributeError: 'list' object has no attribute 'shape'

In [18]:
with open('score.py') as f:
    print(f.read())

import os
import pickle
import json
import joblib
import numpy as np
import torch
from azureml.core.model import Model
from torchvision import transforms
from PIL import Image 

transform = transforms.Compose([            
 transforms.Resize(256),                    
 transforms.CenterCrop(224),                
 transforms.ToTensor(),                     
 transforms.Normalize(                  
 mean=[0.485, 0.456, 0.406],                
 std=[0.229, 0.224, 0.225]                  
 )])


def preprocess_data(img):
    print(f' Image shape in preprocess data: {img.shape}')
    img = Image.fromarray(img, 'RGB')
    print(f' Image type: {type(img)}')
    img_t = transform(img)
    batch_t = torch.unsqueeze(img_t, 0)
    return batch_t

def init():
    global model
    # retrieve the path to the model file using the model name
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'alexnet_model.pkl')
    model = joblib.load(model_path)

def run(raw_data):

    try:
        print(

Deploy your model in the custom environment by providing an [InferenceConfig](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.inferenceconfig?view=azure-ml-py) object to [Model.deploy()](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.model?view=azure-ml-py#deploy-workspace--name--models--inference-config--deployment-config-none--deployment-target-none-). In this case we are also using the [AciWebservice.deploy_configuration()](https://docs.microsoft.com/python/api/azureml-core/azureml.core.webservice.aci.aciwebservice#deploy-configuration-cpu-cores-none--memory-gb-none--tags-none--properties-none--description-none--location-none--auth-enabled-none--ssl-enabled-none--enable-app-insights-none--ssl-cert-pem-file-none--ssl-key-pem-file-none--ssl-cname-none--dns-name-label-none--) method to generate a custom deploy configuration.

**Note**: This step can take several minutes.

In [19]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice


service_name = 'freshen-service'

inference_config = InferenceConfig(entry_script='score.py', environment=environment)
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


After your model is deployed, make a call to the web service using [service.run()](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice%28class%29?view=azure-ml-py#run-input-).

In [22]:
import requests
url = service.scoring_uri
print(url)

http://b790f0dc-11c1-41fb-8ccd-7f1f67567d49.westus.azurecontainer.io/score


In [23]:
import cv2
img = cv2.imread('banana.jpg')
input_data = {'data': img.tolist()}
headers = {'Content-Type':'application/json'}
response = requests.post(url, data=json.dumps(input_data), headers=headers)
print(response)
print(response.json())

<Response [200]>
[-2.45529842376709, -0.4266611337661743, -2.901703119277954, -3.9166030883789062, -2.558084487915039, -1.452687382698059, -2.713717460632324, 2.3359639644622803, -0.3978450298309326, -0.837412416934967, -1.1852779388427734, -0.2879733741283417, -1.3325512409210205, 0.14101053774356842, 0.5280001759529114, -2.145664691925049, -0.4206080138683319, -0.0018470347858965397, -1.1478631496429443, -1.10204017162323, -3.369274377822876, 0.2699413597583771, 1.1452432870864868, 0.08656352013349533, -0.439001202583313, -2.6446993350982666, -2.7630155086517334, -2.320598602294922, -1.617109775543213, -2.1769018173217773, -4.110250473022461, 2.9585084915161133, -0.5302132964134216, -2.114755868911743, -3.821089267730713, -4.410364627838135, -1.9591296911239624, -4.781137943267822, 0.24273177981376648, -2.4248220920562744, 2.380720853805542, -1.5197012424468994, -2.082752227783203, -0.6175000071525574, -0.3882996141910553, -3.5851008892059326, 3.3650999069213867, -0.8800529837608337,

In [1]:
from PIL import Image
img = Image.open('banana.jpg')
img.show

In [30]:
from PIL import Image
print(Image.__version__)

In [27]:
(np.array(response.json())[0])

When you are finished testing your service, clean up the deployment with [service.delete()](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice%28class%29?view=azure-ml-py#delete--).

In [72]:
service.delete()

## Next steps

 - To run a production-ready web service, see the [notebook on deployment to Azure Kubernetes Service](../production-deploy-to-aks/production-deploy-to-aks.ipynb).
 - To run a local web service, see the [notebook on deployment to a local Docker container](../deploy-to-local/register-model-deploy-local.ipynb).
 - For more information on datasets, see the [notebook on training with datasets](../../work-with-data/datasets-tutorial/train-with-datasets/train-with-datasets.ipynb).
 - For more information on environments, see the [notebook on using environments](../../training/using-environments/using-environments.ipynb).
 - For information on all the available deployment targets, see [&ldquo;How and where to deploy models&rdquo;](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-deploy-and-where#choose-a-compute-target).